In [1]:
import pandas as pd
import string
import os
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import urllib
import requests
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
import time
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import folium
from itertools import cycle, islice
from pandas import options
import warnings
import pickle
import re
import nltk
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from matplotlib.pyplot import figure
from nltk.corpus import stopwords 
import nltk


nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


###Here is a Notebook that do the preprocess of our data just like in the first project. I save the splitted data in my drive so as to load them from the other Notebooks to save time.

## *Step 1:* Load and Read Data

In [3]:
path = "/content/drive/My Drive/"

#Read data from csv file 
df = pd.read_csv(path + 'SentimentTweets.csv') 
# df = pd.read_csv(path + 'SentimentTweets.csv',nrows=500000) 
df = df.dropna()
df

,Unnamed: 0,target,id,date,flag,user,text
0,680949,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
1,406741,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
2,1337108,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
3,1560887,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.
4,1466295,4,2064458395,Sun Jun 07 06:19:20 PDT 2009,NO_QUERY,CrazyHan,Completed Race 4 Life in 58mins with girlies f...
...,...,...,...,...,...,...,...
1279995,1385192,4,2052726925,Sat Jun 06 01:47:15 PDT 2009,NO_QUERY,may_oo,@zawhtutwin watching cartoon and cry? oh i do ...
1279996,959516,4,1826067266,Sun May 17 07:37:27 PDT 2009,NO_QUERY,paisss,is eating mcdonalds
1279997,487538,0,2182169058,Mon Jun 15 12:48:49 PDT 2009,NO_QUERY,MayerBabe09,@BestSoyLatte So sorry to hear about your car....
1279998,1575924,4,2189468574,Tue Jun 16 00:06:04 PDT 2009,NO_QUERY,omwayo,@leesherry you have done what you could. Forgi...


#### Take the columns that we want for Sentiment Analysis (text and target columns)

In [4]:
df.drop(df.columns[0], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df.drop(df.columns[1], axis=1, inplace=True)
df

,target,text
0,0,#brokenpromises...
1,0,David Carradine so sad. Thai's law not sure i...
2,4,A @ 415 B @ 425. Tell your bro i say congrats!
3,4,@littlefluffycat Indeed.
4,4,Completed Race 4 Life in 58mins with girlies f...
...,...,...
1279995,4,@zawhtutwin watching cartoon and cry? oh i do ...
1279996,4,is eating mcdonalds
1279997,0,@BestSoyLatte So sorry to hear about your car....
1279998,4,@leesherry you have done what you could. Forgi...


## *Step 2:* Data Cleaning

### a) Lowercase of text set and get rid of the punctiations and symbols.


In [5]:
df['text'] = df['text'].str.lower()

#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    text=re.sub(r'[^a-zA-z0-9\s]','',text)
    return text

df['text'] = df['text'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))

#Apply function on review column
df['text']=df['text'].apply(remove_special_characters)


df

,target,text
0,0,brokenpromises
1,0,david carradine so sad thais law not sure if ...
2,4,a 415 b 425 tell your bro i say congrats
3,4,littlefluffycat indeed
4,4,completed race 4 life in 58mins with girlies f...
...,...,...
1279995,4,zawhtutwin watching cartoon and cry oh i do th...
1279996,4,is eating mcdonalds
1279997,0,bestsoylatte so sorry to hear about your carth...
1279998,4,leesherry you have done what you could forgive...


Observe how many positive and negative comments we have

In [6]:
df['target'].value_counts()

4    640609
0    639391
Name: target, dtype: int64

Replace the negative label of 4 to 1


In [7]:
df['target'] = df['target'].replace(4,1)
display(df['target'])

0          0
1          0
2          1
3          1
4          1
          ..
1279995    1
1279996    1
1279997    0
1279998    1
1279999    0
Name: target, Length: 1280000, dtype: int64

### 2) Remove tags and links 

In [8]:
import re
#Remove tags and links 
tag = re.compile(r'<[^>]+>')

df['text'] = df['text'].apply(lambda x: tag.sub('', x)) #removing html labels

df['text'] = df['text'].replace(r'http\S+', '', regex=True).replace(r'www.\S+', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'"', '', regex=True)

df

,target,text
0,0,brokenpromises
1,0,david carradine so sad thais law not sure if ...
2,1,a 415 b 425 tell your bro i say congrats
3,1,littlefluffycat indeed
4,1,completed race 4 life in 58mins with girlies f...
...,...,...
1279995,1,zawhtutwin watching cartoon and cry oh i do th...
1279996,1,is eating mcdonalds
1279997,0,bestsoylatte so sorry to hear about your carth...
1279998,1,leesherry you have done what you could forgive...


### 3) Tokenization and Lemmanization


In [9]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, pos="v") for w in w_tokenizer.tokenize(text)])

df['text'] = df.text.apply(lemmatize_text).copy()

df

,target,text
0,0,brokenpromises
1,0,david carradine so sad thais law not sure if i...
2,1,a 415 b 425 tell your bro i say congrats
3,1,littlefluffycat indeed
4,1,complete race 4 life in 58mins with girlies fr...
...,...,...
1279995,1,zawhtutwin watch cartoon and cry oh i do that ...
1279996,1,be eat mcdonalds
1279997,0,bestsoylatte so sorry to hear about your carth...
1279998,1,leesherry you have do what you could forgivene...


## *Step 3*: Split dataset into train, test and validation set

I choose to split the dataset to 99%-1% because is too big

In [10]:
x = pd.DataFrame(df, columns = ['text']) 
y = pd.DataFrame(df, columns = ['target']) 

# Split dataset to train and test set.
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.005, random_state=42)

# Split train dataset to train and validation set.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.005, random_state=42)

print("Shape of x_train: ", X_train.shape)
print("Shape of y_train: ", Y_train.shape)
print("Shape of x_test:  ", X_test.shape)
print("Shape of y_test:  ", Y_test.shape)
print("Shape of x_val:  ", X_val.shape)
print("Shape of y_val:  ", Y_val.shape)

Shape of x_train:  (1267232, 1)
Shape of y_train:  (1267232, 1)
Shape of x_test:   (6400, 1)
Shape of y_test:   (6400, 1)
Shape of x_val:   (6368, 1)
Shape of y_val:   (6368, 1)


## *Step 4*: Save the data in order to use them in other Notebooks

In [11]:
x_train_file = open('/content/drive/My Drive/X_train.pkl', 'wb')
pickle.dump(X_train,x_train_file)
x_train_file.close()

x_test_file = open('/content/drive/My Drive/X_test.pkl', 'wb')
pickle.dump(X_test,x_test_file)
x_test_file.close()

y_train_file = open('/content/drive/My Drive/Y_train.pkl', 'wb')
pickle.dump(Y_train,y_train_file)
y_train_file.close()

y_test_file = open('/content/drive/My Drive/Y_test.pkl', 'wb')
pickle.dump(Y_test,y_test_file)
y_test_file.close()

x_val_file = open('/content/drive/My Drive/X_val.pkl', 'wb')
pickle.dump(X_val,x_val_file)
x_val_file.close()

y_val_file = open('/content/drive/My Drive/Y_val.pkl', 'wb')
pickle.dump(Y_val,y_val_file)
y_val_file.close()